In [0]:
!pip install -U git+https://github.com/albu/albumentations
# This lib is required only for training, restart runtime if downloaded

In [0]:
############################### You can skip the training procedure and move directly for testing using pre-trained weights further down below #####################

In [0]:
####################################################### Image pre-processing and augmentation pipeline setup ####################################################

In [0]:
# Enable GPU option in notebook settings

# deep learning and machine learning frameworks
%tensorflow_version 2.x
import pandas as pd
from keras.models import load_model
from keras import backend as K
import tensorflow as tf

# image processing lib
import numpy as np
from numpy.random import permutation
import albumentations as albu
import matplotlib.pyplot as plt
import cv2
import math 
import random
import math

# file management
import os
import sys
from google.colab import drive

In [0]:
# mounting google drive
drive.mount('/content/drive')

In [0]:
# generate npy files for training dataset
# On execution of this cell you will transform the IDRiD training images to 128x128x3 dimensions, and the co-cordinates are also re-scaled to 128x128 image space

# prepare train dataset
fovpath = 'drive/My Drive/FundusPoseNet/IDriD/C. Localization/2. Groundtruths/2. Fovea Center Location/IDRiD_Fovea_Center_Training Set_Markups.csv'
odpath = 'drive/My Drive/FundusPoseNet/IDriD/C. Localization/2. Groundtruths/1. Optic Disc Center Location/a. IDRiD_OD_Center_Training Set_Markups.csv'
imglist = os.listdir("drive/My Drive/FundusPoseNet/IDriD/C. Localization/1. Original Images/a. Training Set")
imgpath = "drive/My Drive/FundusPoseNet/IDriD/C. Localization/1. Original Images/a. Training Set/{}"

fov = pd.read_csv(fovpath, encoding='utf-8')
od = pd.read_csv(odpath, encoding='utf-8')

print(len(fov), len(od), len(imglist))

x = []
y = np.zeros((len(imglist), 4))
S = 128 # image resolution of generated npy files
for p in enumerate(imglist):
  i = p[0]
  p = p[1]
  img = cv2.imread(imgpath.format(p))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  H, W = img.shape[0], img.shape[1] 
  o = od.loc[od['Image No'] == p.split('.')[0]].iloc[0]
  f = fov.loc[fov['Image No'] == p.split('.')[0]].iloc[0]
  # scale the dimension     
  y[i] = [o['X- Coordinate']/W*S, o['Y - Coordinate']/H*S,
          f['X- Coordinate']/W*S, f['Y - Coordinate']/H*S]  
  x.append(cv2.resize(img, (S, S), interpolation=cv2.INTER_AREA))

x = np.array(x)
print(x.shape, y.shape)
print(x.max(), y.max())
print(x.min(), y.min())

# save train data 
np.save('drive/My Drive/FundusPoseNet/npy/x_train.npy', x)
np.save('drive/My Drive/FundusPoseNet/npy/y_train.npy', y)

################################################################################################################################

# prepare val dataset (we used IDRiD test set as val set during training)
fovpath = 'drive/My Drive/FundusPoseNet/IDriD/C. Localization/2. Groundtruths/2. Fovea Center Location/IDRiD_Fovea_Center_Testing Set_Markups.csv'
odpath = 'drive/My Drive/FundusPoseNet/IDriD/C. Localization/2. Groundtruths/1. Optic Disc Center Location/b. IDRiD_OD_Center_Testing Set_Markups.csv'
imglist = os.listdir("drive/My Drive/FundusPoseNet/IDriD/C. Localization/1. Original Images/b. Testing Set")
imgpath = "drive/My Drive/FundusPoseNet/IDriD/C. Localization/1. Original Images/b. Testing Set/{}"

fov = pd.read_csv(fovpath, encoding='utf-8')
od = pd.read_csv(odpath, encoding='utf-8')

print(len(fov), len(od), len(imglist))

x = []
y = np.zeros((len(imglist), 4))
S = 128 # image resolution of generated npy files
for p in enumerate(imglist):
  i = p[0]
  p = p[1]
  img = cv2.imread(imgpath.format(p))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  H, W = img.shape[0], img.shape[1] 
  o = od.loc[od['Image No'] == p.split('.')[0]].iloc[0]
  f = fov.loc[fov['Image No'] == p.split('.')[0]].iloc[0]
  # scale the dimension     
  y[i] = [o['X- Coordinate']/W*S, o['Y - Coordinate']/H*S,
          f['X- Coordinate']/W*S, f['Y - Coordinate']/H*S]  
  x.append(cv2.resize(img, (S, S), interpolation=cv2.INTER_AREA))

x = np.array(x)
print(x.shape, y.shape)
print(x.max(), y.max())
print(x.min(), y.min())

# save val data 
np.save('drive/My Drive/FundusPoseNet/npy/x_val.npy', x)
np.save('drive/My Drive/FundusPoseNet/npy/y_val.npy', y)

In [0]:
# load data
x_train = np.load("drive/My Drive/FundusPoseNet/npy/x_train.npy")/255.0
y_train = np.load("drive/My Drive/FundusPoseNet/npy/y_train.npy")
x_val = np.load("drive/My Drive/FundusPoseNet/npy/x_val.npy")/255.0
y_val = np.load("drive/My Drive/FundusPoseNet/npy/y_val.npy")

# shuffle the images
perm = np.random.permutation(len(x_train))
x_train = x_train[perm]
y_train = y_train[perm]

# log important data about pre-processed data
print(x_train.shape, y_train.shape)
print(x_train.max(), y_train.max())
print(x_train.min(), y_train.min())
print(x_val.shape, y_val.shape)
print(x_val.max(), y_val.max())
print(x_val.min(), y_val.min())

In [0]:
# image augmentation pipeline
# A batch of image is sampled from x_train and heatmaps are generated during the runtime by the augmentor() generator

def gaussian_heatmap(x0, y0, isFov, width=128, height=128):
  """ Make a square gaussian kernel centered at (x0, y0) with sigma as SD.
  """
  if isFov == 1:
    # for fovea sigma is selected as 10.0
    sigma = 10.0
  else:
    # for optic disc sigma is selected as 8.0
    sigma = 8.0    
  x = np.arange(0, width, 1, float)
  y = np.arange(0, height, 1, float)[:, np.newaxis]
  # generate and return the heatmap
  return np.exp(-((x-x0)**2 + (y-y0)**2) / (2*sigma**2))

def apply_transforms(transformations, image, points):
  return albu.Compose(transformations, p=0.5, 
                      keypoint_params=albu.KeypointParams(format='xy'))(image=image, keypoints=points)

def augmentor(batch):
  while True:  
    batch_x, batch_y = next(batch) # obtain a batch
    IMGS = np.zeros((batch_x.shape[0], batch_x.shape[1], batch_x.shape[2], batch_x.shape[3])) # placeholder for transformed image
    MASKS = np.zeros((batch_x.shape[0], batch_x.shape[1], batch_x.shape[2], batch_y.shape[1]//2)) # placeholder for masks   
    for data in enumerate(zip(batch_x, batch_y)):
      img = data[1][0] # image
      kvec = data[1][1] # keypoints
      kps = np.array([(kvec[k], kvec[k+1]) for k in range(0, batch_y.shape[1]-1, 2)])
      res = apply_transforms([albu.VerticalFlip(p=0.5),
                              albu.HorizontalFlip(p=0.5),
                              albu.ShiftScaleRotate(p=0.5, shift_limit=0.0, rotate_limit=0, scale_limit=0.4, border_mode=cv2.BORDER_CONSTANT)],
                              img, kps)
      
      IMGS[data[0]] = res['image']
      for k in range(0, batch_y.shape[1]//2):
        try:
          MASKS[data[0], :, :, k] = gaussian_heatmap(res['keypoints'][k][0], res['keypoints'][k][1], k)
        except:
          IMGS[data[0]] = IMGS[0]
          MASKS[data[0]] = MASKS[0]
          print("Error occured in augmentation")  
    del batch_x, batch_y
    # generates a batch of augmented images and labels during runtime of training 
    yield (IMGS, MASKS)

# split data for training and validation in 9:1 ratio 
SEED=None
BATCH_SIZE=12
KPS = 2
C = 3
IMG_SIZE=128

data_gen_args = dict(brightness_range=[0.5, 1.5],
                     rescale=1./255)
gen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
# prepare the train and val generators
train_gen = augmentor(gen.flow(x_train, y_train, batch_size=BATCH_SIZE, seed=SEED)) 
val_gen = augmentor(gen.flow(x_val, y_val, batch_size=BATCH_SIZE, seed=SEED)) 

In [0]:
# sample batch of data from the augmentation pipeline to verify if images are augmented as required
for imgs, masks in train_gen:
  print(masks.max(), imgs.max())
  print(masks.shape, imgs.shape)
  for img, mask in zip(imgs, masks):
    for k in range(KPS):
      plt.axis('off')
      plt.imshow(mask[:, :, k], cmap='gray')
      plt.show()
    m = np.sum(mask, axis=-1, keepdims=True)
    plt.axis('off')
    plt.imshow(img) 
    plt.show()
    plt.axis('off')
    plt.imshow(m*img)
    plt.show()    
    break
  break

In [0]:
##################################################################### Model Architecture and training ####################################################

In [0]:
# encoder and decoder blocks
def Encoder(x, k, ks, dr, a=0.1):
  x = tf.keras.layers.Conv2D(2 * k, kernel_size=(ks, ks), strides=(1, 1), padding='same', dilation_rate=(dr, dr))(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU(alpha=a)(x)
  x = tf.keras.layers.Conv2D(k, kernel_size=(ks, ks), strides=(1, 1), padding='same', dilation_rate=(dr, dr))(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU(alpha=a)(x)
  x = tf.keras.layers.Conv2D(2 * k, kernel_size=(ks, ks), strides=(1, 1), padding='same', dilation_rate=(dr, dr))(x) 
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU(alpha=a)(x)  
  pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))(x)
  return x, pool

def Decoder(x, skip, k, ks, dr, a=0.1):
  x = tf.keras.layers.concatenate([tf.keras.layers.Conv2DTranspose(k, kernel_size=(2, 2), strides=(2, 2), padding='same')(x), skip])
  x = tf.keras.layers.Conv2D(2 * k, kernel_size=(ks, ks), strides=(1, 1), padding='same', dilation_rate=(dr, dr))(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU(alpha=a)(x)
  x = tf.keras.layers.Conv2D(k, kernel_size=(ks, ks), strides=(1, 1), padding='same', dilation_rate=(dr, dr))(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU(alpha=a)(x)
  x = tf.keras.layers.Conv2D(2 * k, kernel_size=(ks, ks), strides=(1, 1), padding='same', dilation_rate=(dr, dr))(x) 
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU(alpha=a)(x)  
  return x

def BuildModel(k, IMG_SIZE=128, KPS=2, C=3):
  inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, C))
  conv1, pool1 = Encoder(inputs, k, 7, 1)
  k = k * 2
  conv2, pool2 = Encoder(pool1, k, 5, 1)
  k = k * 2
  conv3, pool3 = Encoder(pool2, k, 3, 1)
  k = k * 2
  conv4, pool4 = Encoder(pool3, k, 3, 2)
  k = k * 2
  conv5, _ = Encoder(pool4, k, 3, 2)
  deconv1 = Decoder(conv5, conv4, k, 3, 2)
  k = k // 2
  deconv2 = Decoder(deconv1, conv3, k, 3, 1)
  k = k // 2
  deconv3 = Decoder(deconv2, conv2, k, 5, 1)
  k = k // 2
  deconv4 = Decoder(deconv3, conv1, k, 7, 1)
  outputs = tf.keras.layers.Conv2D(KPS, kernel_size=(1, 1))(deconv4)
  outputs = tf.keras.layers.BatchNormalization()(outputs)
  outputs = tf.keras.layers.Activation('sigmoid')(outputs)
  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  return model   

In [0]:
# build the model
fp = BuildModel(16)
fp.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),
           loss='binary_crossentropy')

# uncomment the following line, to continue training from last checkpoint
fp.summary()

# checkpoint to save the model architecture, optimizer state and model weights and biases
checkpoint = tf.keras.callbacks.ModelCheckpoint("drive/My Drive/FundusPoseNet/models/FPN.model",
                                                monitor='val_loss', # monitor the val loss
                                                verbose=1,
                                                save_best_only=True,
                                                save_weights_only=False,
                                                mode='min', # save model if val loss is less
                                                period=1)

In [0]:
# train the model
history = fp.fit_generator(train_gen,
                 steps_per_epoch=x_train.shape[0] // BATCH_SIZE,
                 epochs=400,
                 verbose=1,
                 callbacks=[checkpoint],
                 validation_data=val_gen,
                 validation_steps=x_val.shape[0] // BATCH_SIZE,
                 shuffle=True)   

In [0]:
# plot curves
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

plt.plot(history.history['loss'])
plt.ylabel('train_loss')
plt.xlabel('epoch')
plt.show()

plt.plot(history.history['val_loss'])
plt.ylabel('val_loss')
plt.xlabel('epoch')
plt.show()

In [0]:
######################################################################### Testing and intermediate layer analysis #############################################################

In [0]:
# Uncomment line 2 to load the pre-trained model
# model = tf.keras.models.load_model("drive/My Drive/FundusPoseNet/models/FundusPoseNet128") # ignore any warnings
# Uncomment line 4 to load the model you just trained 
model = tf.keras.models.load_model("drive/My Drive/FundusPoseNet/models/FPN.model") # re-run cell 2 (mount the drive again if it thorws any file not found error)
IMG_SIZE = 128 # input image size

In [0]:
################################################################ Test the model on IDRiD test dataset and view the results ##################################################### 

In [0]:
def EDprediction(paths, imgpath):
  count = 0
  predictedPos = []
  imName = []
  print(len(paths))
  for p in paths:
    try:
      # pre-process raw image
      img = cv2.imread(imgpath.format(p))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      # find the dimension re-scaling factor
      rH, rW = (1. * img.shape[0])/(1. * IMG_SIZE), (1. * img.shape[1])/(1. * IMG_SIZE)
      # resize the image 
      img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)/255.0
      # feed the input image
      output = model.predict([img.reshape(1, IMG_SIZE, IMG_SIZE, 3)])   
      points = []
      for i in range(2): 
        # apply gaussian blur using 25x25 kerenl
        blur = cv2.GaussianBlur(np.uint8(output[0, :, :, i]*255.0), (25, 25), 0)
        # apply otsu's thresholding
        _,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        # detect the countours    
        cnts, _ = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        max_area_index = [cv2.contourArea(cnt) for cnt in cnts]
        index = max_area_index.index(max(max_area_index))
        # apply moments on contour with max area
        M = cv2.moments(cnts[index])
        # find the co-ordinates from moments 
        cx = M['m10']/M['m00']
        cy = M['m01']/M['m00']
        points.append(cx*rW)
        points.append(cy*rH) 
      predictedPos.append(points)
      imName.append(p.split(".")[0])
      count = count + 1
    except:
      print(p)  
  print(count)
  predictedPos = np.array(predictedPos)
  return predictedPos, imName

In [0]:
# load test data
paths = os.listdir("drive/My Drive/FundusPoseNet/IDriD/C. Localization/1. Original Images/b. Testing Set")
fovpath = 'drive/My Drive/FundusPoseNet/IDriD/C. Localization/2. Groundtruths/2. Fovea Center Location/IDRiD_Fovea_Center_Testing Set_Markups.csv'
odpath = 'drive/My Drive/FundusPoseNet/IDriD/C. Localization/2. Groundtruths/1. Optic Disc Center Location/b. IDRiD_OD_Center_Testing Set_Markups.csv'
imgpath = "drive/My Drive/FundusPoseNet/IDriD/C. Localization/1. Original Images/b. Testing Set/{}"

fov = pd.read_csv(fovpath, encoding='utf-8')
od = pd.read_csv(odpath, encoding='utf-8')

res, imName = EDprediction(paths[:5], imgpath)
print(res.shape)

In [0]:
# get euclidean distance
edfov = 0.0
edod = 0.0
count = 0 

# blue dot ground truth, red dot is predicted co-ordinate
for name in imName:
  img = cv2.imread("drive/My Drive/FundusPoseNet/IDriD/C. Localization/1. Original Images/b. Testing Set/{}.jpg".format(name))
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  x = fov.loc[fov['Image No'] == name].iloc[0]
  pos = imName.index(name)  
  px, py = x['X- Coordinate'], x['Y - Coordinate'] 
  qx, qy = res[pos][2], res[pos][3]
  cv2.circle(img, (int(px), int(py)), 20, (255,0,0), -1)
  cv2.circle(img, (int(qx), int(qy)), 20, (0,0,255), -1)  
  edfov = edfov + math.sqrt(math.pow((px-qx), 2)+math.pow((py-qy), 2))
  print("edfov: {}".format(math.sqrt(math.pow((px-qx), 2)+math.pow((py-qy), 2))))

  x = od.loc[od['Image No'] == name].iloc[0]
  pos = imName.index(name)  
  px, py = x['X- Coordinate'], x['Y - Coordinate'] 
  qx, qy = res[pos][0], res[pos][1]
  cv2.circle(img, (int(px), int(py)), 20, (255,0,0), -1)
  cv2.circle(img, (int(qx), int(qy)), 20, (0,0,255), -1)
  edod = edod + math.sqrt(math.pow((px-qx), 2)+math.pow((py-qy), 2))  
  plt.imshow(img)
  plt.show()
  print("edod: {}".format(math.sqrt(math.pow((px-qx), 2)+math.pow((py-qy), 2))))
  count = count + 1

In [0]:
# display the final score
print(edfov, edod, count)  
print(edfov/count, edod/count)

In [0]:
############################################################## Test on one image to view intermediate layers ##################################################################

In [0]:
# single image output testing and layer by layer analysis
fp.set_weights(model.get_weights()) 
inter = [tf.keras.Model(inputs=fp.input, outputs=fp.layers[i].output) for i in range(1, len(model.layers))]

imgpath = 'drive/My Drive/FundusPoseNet/IDriD/C. Localization/1. Original Images/b. Testing Set/IDRiD_001.jpg.jpg'

img = cv2.imread(imgpath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
rH, rW = (1. * img.shape[0])/(1. * IMG_SIZE), (1. * img.shape[1])/(1. * IMG_SIZE) 
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)/255.0
output = model.predict([img.reshape(1, IMG_SIZE, IMG_SIZE, 3)])
tmp = fp.predict([img.reshape(1, IMG_SIZE, IMG_SIZE, 3)])
try:
  for i in range(2, len(model.layers)):
    print('Layer {}'.format(i))
    o = inter[i].predict([img.reshape(1, IMG_SIZE, IMG_SIZE, 3)])
    for j in range(0, o.shape[3], 50): # better keep step=50, otherwise it takes forever
      print('Channel {}'.format(j))
      plt.imshow(o[0, :, :, j], cmap='gray')
      plt.show()
except:
  pass         
points = []
for i in range(2):
  plt.imshow(output[0, :, :, i], cmap='gray')
  plt.show() 
  blur = cv2.GaussianBlur(np.uint8(output[0, :, :, i]*255.0), (25, 25), 0)
  _,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)    
  cnts, _ = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  max_area_index = [cv2.contourArea(cnt) for cnt in cnts]
  index = max_area_index.index(max(max_area_index))
  M = cv2.moments(cnts[index])
  cx = M['m10']/M['m00']
  cy = M['m01']/M['m00']
  img = cv2.drawMarker(img, (int(cx), int(cy)), (255,255,0), markerType=cv2.MARKER_CROSS, 
                  markerSize=5, thickness=1, line_type=cv2.LINE_AA)

plt.imshow(img)
plt.show()     

In [0]:
################################################################# IGNORE THIS #########################################################################

# # save intermediate steps for presenatation/illustration on paper 
# save_name_template = ['drive/My Drive/Output/{}/od_heatmap.jpg',
#                       'drive/My Drive/Output/{}/fov_heatmap.jpg',
#                       'drive/My Drive/Output/{}/overlay.jpg',
#                       'drive/My Drive/Output/{}/dot_plot.jpg',
#                       'drive/My Drive/Output/{}/crosshair.jpg',
#                       'drive/My Drive/Output/{}/overlay_dot.jpg',
#                       'drive/My Drive/Output/{}/overlay_crosshair.jpg']

# for i in range(100):
#   p = []
#   img = cv2.imread('drive/My Drive/Output/{}/{}'.format(i, os.listdir('drive/My Drive/Output/{}'.format(i))[0]))
#   img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#   img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)/255.0
#   imgo = img.copy()
#   imgo = cv2.cvtColor(np.uint8(imgo*255), cv2.COLOR_RGB2BGR)
#   output = model.predict([img.reshape(1, IMG_SIZE, IMG_SIZE, 3)])   
#   points = []
#   pp = imgo.copy()
#   pc = imgo.copy()
#   o = np.sum(output, axis=-1).reshape(128, 128, 1)
#   o = o*imgo
#   outputp = o.copy()
#   outputc = o.copy()
#   cv2.imwrite(save_name_template[0].format(i), output[0, :, :, 0]*255)
#   cv2.imwrite(save_name_template[1].format(i), output[0, :, :, 1]*255)
#   for j in range(2):
#     #plt.imshow(output[0, :, :, j], cmap='gray')
#     #plt.show() 
#     blur = cv2.GaussianBlur(np.uint8(output[0, :, :, j]*255.0), (25, 25), 0)
#     _,th = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)    
#     cnts, _ = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     max_area_index = [cv2.contourArea(cnt) for cnt in cnts]
#     index = max_area_index.index(max(max_area_index))
#     M = cv2.moments(cnts[index])
#     cx = M['m10']/M['m00']
#     cy = M['m01']/M['m00']
#     cv2.circle(pp, (int(cx), int(cy)), 1, (255,255,0), -1) 
#     pc = cv2.drawMarker(pc, (int(cx), int(cy)), (255,255,0), markerType=cv2.MARKER_CROSS, 
#                    markerSize=5, thickness=1, line_type=cv2.LINE_AA) 
#     cv2.circle(outputp, (int(cx), int(cy)), 1, (255,255,0), -1) 
#     outputc = cv2.drawMarker(outputc, (int(cx), int(cy)), (255,255,0), markerType=cv2.MARKER_CROSS, 
#                    markerSize=5, thickness=1, line_type=cv2.LINE_AA)     
#     p.append(points)

  
#   #plt.imshow(o)
#   #plt.show()
#   cv2.imwrite(save_name_template[2].format(i), o)

#   #plt.imshow(outputp)
#   #plt.show()
#   cv2.imwrite(save_name_template[5].format(i), outputp)

#   #plt.imshow(outputc)
#   #plt.show()
#   cv2.imwrite(save_name_template[6].format(i), outputc)

#   #plt.imshow(pp)
#   #plt.show()
#   cv2.imwrite(save_name_template[3].format(i), pp)

#   #plt.imshow(pc)
#   #plt.show()
#   cv2.imwrite(save_name_template[4].format(i), pc)